# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [9]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
cities = cities.replace('\[.*?\]', '', regex = True)
NHL_cities = cities[['Metropolitan area', 'Population', 'NHL']]
NHL_cities.head()


,Metropolitan area,Population,NHL
0,New York City,20153634,RangersIslandersDevils
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [24]:
NHL_team = []
NHL_col = []
regex = '[\d|A-Z]+[\d|a-z|\s]+'
for entry in NHL_cities['NHL']:
    x = re.findall(regex, entry)
    NHL_team = NHL_team + x
    NHL_col.append(x)
NHL_cities['NHL_teams'] = NHL_col
NHL_cities.head()


    

['Rangers', 'Islanders', 'Devils', 'Kings', 'Ducks', 'Sharks', 'Blackhawks', 'Stars', 'Capitals', 'Flyers', 'Bruins', 'Wild', 'Avalanche', 'Panthers', 'Coyotes', 'Red ', 'Wings', 'Maple ', 'Leafs', 'Lightning', 'Penguins', 'Blues', 'Predators', 'Sabres', 'Canadiens', 'Canucks', 'Blue ', 'Jackets', 'Flames', 'Senators', 'Oilers', 'Jets', 'Golden ', 'Knights', 'Hurricanes']


,Metropolitan area,Population,NHL,NHL_teams
0,New York City,20153634,RangersIslandersDevils,"[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,KingsDucks,"[Kings, Ducks]"
2,San Francisco Bay Area,6657982,Sharks,[Sharks]
3,Chicago,9512999,Blackhawks,[Blackhawks]
4,Dallas–Fort Worth,7233323,Stars,[Stars]


In [30]:
def clean(team_list):
    return_list = []
    current = ""
    for team in team_list:
        if team[-1] != ' ':
            current = current + team
            return_list.append(current)
            current = ''
        else:
            current = current + team
    return return_list

NHL_team = clean(NHL_team)
            

In [40]:
def clean_list(team_list):
    if len(team_list) > 0:
        return_list = []
        current = ''
        for team in team_list:
            if team[-1] != ' ':
                current = current + team
                return_list.append(current)
                current = ''
            else:
                current = current + team
        return return_list
    else:
        return []

NHL_cities['final_list'] = NHL_cities.apply(lambda x: clean_list(x['NHL_teams']), axis = 1)
NHL_cities.head(20)

,Metropolitan area,Population,NHL,NHL_teams,final_list
0,New York City,20153634,RangersIslandersDevils,"[Rangers, Islanders, Devils]","[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,KingsDucks,"[Kings, Ducks]","[Kings, Ducks]"
2,San Francisco Bay Area,6657982,Sharks,[Sharks],[Sharks]
3,Chicago,9512999,Blackhawks,[Blackhawks],[Blackhawks]
4,Dallas–Fort Worth,7233323,Stars,[Stars],[Stars]
5,"Washington, D.C.",6131977,Capitals,[Capitals],[Capitals]
6,Philadelphia,6070500,Flyers,[Flyers],[Flyers]
7,Boston,4794447,Bruins,[Bruins],[Bruins]
8,Minneapolis–Saint Paul,3551036,Wild,[Wild],[Wild]
9,Denver,2853077,Avalanche,[Avalanche],[Avalanche]


In [31]:
def NHL_team_parse(team):
    for i in range(len(team)):
        if team[i] == ' ':
            possible_team = team[i+1:]
            for example in NHL_team:
                if possible_team == example:
                    return possible_team
    return False

In [33]:
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df = nhl_df.replace('\*$', '', regex = True)
nhl_df = nhl_df[['team', 'W', 'L']]
numeric = nhl_df.apply(lambda x: x['W'].isnumeric(), axis = 1)
nhl_df = nhl_df[numeric]
nhl_df.head()


,team,W,L
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30
5,Detroit Red Wings,30,39


In [35]:
nhl_df['team parsed'] = nhl_df.apply(lambda x: NHL_team_parse(x['team']), axis = 1)
nhl_df.head()

,team,W,L,team parsed
1,Tampa Bay Lightning,54,23,Lightning
2,Boston Bruins,50,20,Bruins
3,Toronto Maple Leafs,49,26,Maple Leafs
4,Florida Panthers,44,30,Panthers
5,Detroit Red Wings,30,39,Red Wings


In [22]:
def NHL_clean(team_list):
    if len(team_list) > 0:
        return_list = []
        current = ''
        for team in team_list:
            if team[-1] != ' ':
                current = current + team
                return_list.append(current)
                current = ''
            else:
                current = current + team
        return return_list
    else:
        return []

def NHL_team_parse(team, NHL_team):
    for i in range(len(team)):
        if team[i] == ' ':
            possible_team = team[i+1:]
            for example in NHL_team:
                if possible_team == example:
                    return possible_team
    return False

def nhl_win_loss(team_list, nhl_df):
    win = 0
    loss = 0
    if len(team_list) > 0:
        for team in team_list:
            win_stat = nhl_df.loc[team, 'W']
            loss_stat = nhl_df.loc[team, 'L']
            win += int(win_stat)
            loss += int(loss_stat)
        return win/(win+loss)
    else:
        return False
    
    

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#nhl_df=pd.read_csv("assets/nhl.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'NHL']]
    
    # Parse team data from cities dataframe    
    nhl_team = []
    nhl_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['NHL']:
        x = re.findall(regex, entry)
        nhl_team = nhl_team + x
        nhl_col.append(x)
    cities['NHL_teams'] = nhl_col
    
    nhl_team = NHL_clean(nhl_team)
    cities['NHL_teams_cleaned'] =  cities.apply(lambda x: NHL_clean(x['NHL_teams']), axis = 1)
    
    # Clean nhl df
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df.replace('\*$', '', regex = True)
    nhl_df = nhl_df[['team', 'W', 'L']]
    numeric = nhl_df.apply(lambda x: x['W'].isnumeric(), axis = 1)
    nhl_df = nhl_df[numeric]
    nhl_df['team parsed'] = nhl_df.apply(lambda x: NHL_team_parse(x['team'], nhl_team), axis = 1)
    nhl_df = nhl_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio'] = cities.apply(lambda x: nhl_win_loss(x['NHL_teams_cleaned'], nhl_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio'] != False]
        
    
    population_by_region = [int(x) for x in list(cities['Population'])] # pass in metropolitan area population from cities
    win_loss_by_region = list(cities['W_L ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [45]:
def NBA_clean(team_list):
    if len(team_list) > 0:
        return_list = []
        current = ''
        for team in team_list:
            if team[-1] != ' ':
                current = current + team
                return_list.append(current)
                current = ''
            else:
                current = current + team
        return return_list
    else:
        return []

def NBA_team_parse(team, NBA_team):
    for i in range(len(team)):
        if team[i] == ' ':
            possible_team = team[i+1:].strip()
            for example in NBA_team:
                if possible_team == example:
                    return possible_team
    return False

def nba_win_loss(team_list, nba_df):
    win = 0
    loss = 0
    if len(team_list) > 0:
        for team in team_list:
            win_stat = nba_df.loc[team, 'W']
            loss_stat = nba_df.loc[team, 'L']
            win += int(win_stat)
            loss += int(loss_stat)
        return win/(win+loss)
    else:
        return False

In [46]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'NBA']]
    
    # Parse team data from cities dataframe    
    nba_team = []
    nba_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['NBA']:
        x = re.findall(regex, entry)
        nba_team = nba_team + x
        nba_col.append(x)
    cities['NBA_teams'] = nba_col
    
    nba_team = NBA_clean(nba_team)
    cities['NBA_teams_cleaned'] =  cities.apply(lambda x: NBA_clean(x['NBA_teams']), axis = 1)
    
    # Clean nba df
    nba_df = nba_df[nba_df['year'] == 2018]
    pre_regex = '^[A-Z|\d|a-z|\s]+'
    nba_df['team'] = nba_df.apply(lambda x: re.findall(pre_regex, x['team'])[0], axis = 1)
    nba_df = nba_df[['team', 'W', 'L']]
    nba_df['team parsed'] = nba_df.apply(lambda x: NBA_team_parse(x['team'], nba_team), axis = 1)
    nba_df = nba_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio'] = cities.apply(lambda x: nba_win_loss(x['NBA_teams_cleaned'], nba_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio'] != False]    
    
    population_by_region = [int(x) for x in list(cities['Population'])] # pass in metropolitan area population from cities
    win_loss_by_region = list(cities['W_L ratio']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [52]:
def MLB_clean(team_list):
    if len(team_list) > 0:
        return_list = []
        current = ''
        for team in team_list:
            if team[-1] != ' ':
                current = current + team
                return_list.append(current)
                current = ''
            else:
                current = current + team
        return return_list
    else:
        return []

def MLB_team_parse(team, MLB_team):
    for i in range(len(team)):
        if team[i] == ' ':
            possible_team = team[i+1:].strip()
            for example in MLB_team:
                if possible_team == example:
                    return possible_team
    return False

def mlb_win_loss(team_list, mlb_df):
    win = 0
    loss = 0
    if len(team_list) > 0:
        for team in team_list:
            win_stat = mlb_df.loc[team, 'W']
            loss_stat = mlb_df.loc[team, 'L']
            win += int(win_stat)
            loss += int(loss_stat)
        return win/(win+loss)
    else:
        return False

In [54]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'MLB']]
    
    # Parse team data from cities dataframe    
    mlb_team = []
    mlb_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['MLB']:
        x = re.findall(regex, entry)
        mlb_team = mlb_team + x
        mlb_col.append(x)
    cities['MLB_teams'] = mlb_col
    
    mlb_team = MLB_clean(mlb_team)
    cities['MLB_teams_cleaned'] =  cities.apply(lambda x: MLB_clean(x['MLB_teams']), axis = 1)
    
    # Clean mlb df
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df = mlb_df[['team', 'W', 'L']]
    mlb_df['team parsed'] = mlb_df.apply(lambda x: MLB_team_parse(x['team'], mlb_team), axis = 1)
    mlb_df = mlb_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio'] = cities.apply(lambda x: mlb_win_loss(x['MLB_teams_cleaned'], mlb_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio'] != False]
    
    population_by_region = [int(x) for x in list(cities['Population'])] # pass in metropolitan area population from cities
    win_loss_by_region = list(cities['W_L ratio']) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [59]:
def NFL_clean(team_list):
    if len(team_list) > 0:
        return_list = []
        current = ''
        for team in team_list:
            if team[-1] != ' ':
                current = current + team
                return_list.append(current)
                current = ''
            else:
                current = current + team
        return return_list
    else:
        return []
    
def NFL_team_parse(team, NFL_team):
    for i in range(len(team)):
        if team[i] == ' ':
            possible_team = team[i+1:].strip()
            for example in NFL_team:
                if possible_team == example:
                    return possible_team
    return False

def nfl_win_loss(team_list, nfl_df):
    win = 0
    loss = 0
    if len(team_list) > 0:
        for team in team_list:
            win_stat = nfl_df.loc[team, 'W']
            loss_stat = nfl_df.loc[team, 'L']
            win += int(win_stat)
            loss += int(loss_stat)
        return win/(win+loss)
    else:
        return False

In [60]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'NFL']]
    
    # Parse team data from cities dataframe    
    nfl_team = []
    nfl_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['NFL']:
        x = re.findall(regex, entry)
        nfl_team = nfl_team + x
        nfl_col.append(x)
    cities['NFL_teams'] = nfl_col
    
    nfl_team = NFL_clean(nfl_team)
    cities['NFL_teams_cleaned'] =  cities.apply(lambda x: NFL_clean(x['NFL_teams']), axis = 1)
    
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df.replace('[\*|\+]$', '', regex = True)
    nfl_df = nfl_df[['team', 'W', 'L']]
    numeric = nfl_df.apply(lambda x: x['W'].isnumeric(), axis = 1)
    nfl_df = nfl_df[numeric]
    nfl_df['team parsed'] = nfl_df.apply(lambda x: NFL_team_parse(x['team'], nfl_team), axis = 1)
    nfl_df = nfl_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio'] = cities.apply(lambda x: nfl_win_loss(x['NFL_teams_cleaned'], nfl_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio'] != False]
    
    
    population_by_region = [int(x) for x in list(cities['Population'])] # pass in metropolitan area population from cities
    win_loss_by_region = list(cities['W_L ratio']) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [61]:
def nhl_gen(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'NHL']]
    
    # Parse team data from cities dataframe    
    nhl_team = []
    nhl_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['NHL']:
        x = re.findall(regex, entry)
        nhl_team = nhl_team + x
        nhl_col.append(x)
    cities['NHL_teams'] = nhl_col
    
    nhl_team = NHL_clean(nhl_team)
    cities['NHL_teams_cleaned'] =  cities.apply(lambda x: NHL_clean(x['NHL_teams']), axis = 1)
    
    # Clean nhl df
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df.replace('\*$', '', regex = True)
    nhl_df = nhl_df[['team', 'W', 'L']]
    numeric = nhl_df.apply(lambda x: x['W'].isnumeric(), axis = 1)
    nhl_df = nhl_df[numeric]
    nhl_df['team parsed'] = nhl_df.apply(lambda x: NHL_team_parse(x['team'], nhl_team), axis = 1)
    nhl_df = nhl_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio_NHL'] = cities.apply(lambda x: nhl_win_loss(x['NHL_teams_cleaned'], nhl_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio_NHL'] != False]
    
    return cities[['W_L ratio_NHL']]

In [62]:
def nba_gen():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'NBA']]
    
    # Parse team data from cities dataframe    
    nba_team = []
    nba_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['NBA']:
        x = re.findall(regex, entry)
        nba_team = nba_team + x
        nba_col.append(x)
    cities['NBA_teams'] = nba_col
    
    nba_team = NBA_clean(nba_team)
    cities['NBA_teams_cleaned'] =  cities.apply(lambda x: NBA_clean(x['NBA_teams']), axis = 1)
    
    # Clean nba df
    nba_df = nba_df[nba_df['year'] == 2018]
    pre_regex = '^[A-Z|\d|a-z|\s]+'
    nba_df['team'] = nba_df.apply(lambda x: re.findall(pre_regex, x['team'])[0], axis = 1)
    nba_df = nba_df[['team', 'W', 'L']]
    nba_df['team parsed'] = nba_df.apply(lambda x: NBA_team_parse(x['team'], nba_team), axis = 1)
    nba_df = nba_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio_NBA'] = cities.apply(lambda x: nba_win_loss(x['NBA_teams_cleaned'], nba_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio_NBA'] != False]    

    return cities[['W_L ratio_NBA']]

In [63]:
def mlb_gen(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'MLB']]
    
    # Parse team data from cities dataframe    
    mlb_team = []
    mlb_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['MLB']:
        x = re.findall(regex, entry)
        mlb_team = mlb_team + x
        mlb_col.append(x)
    cities['MLB_teams'] = mlb_col
    
    mlb_team = MLB_clean(mlb_team)
    cities['MLB_teams_cleaned'] =  cities.apply(lambda x: MLB_clean(x['MLB_teams']), axis = 1)
    
    # Clean mlb df
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df = mlb_df[['team', 'W', 'L']]
    mlb_df['team parsed'] = mlb_df.apply(lambda x: MLB_team_parse(x['team'], mlb_team), axis = 1)
    mlb_df = mlb_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio_MLB'] = cities.apply(lambda x: mlb_win_loss(x['MLB_teams_cleaned'], mlb_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio_MLB'] != False]

    return cities[['W_L ratio_MLB']]

In [64]:
def nfl_gen(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Clean city dataframe
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    cities = cities.replace('\[.*?\]', '', regex = True)
    cities = cities[['Metropolitan area', 'Population', 'NFL']]
    
    # Parse team data from cities dataframe    
    nfl_team = []
    nfl_col = []
    regex = '[\d|A-Z]+[\d|a-z|\s]+'
    for entry in cities['NFL']:
        x = re.findall(regex, entry)
        nfl_team = nfl_team + x
        nfl_col.append(x)
    cities['NFL_teams'] = nfl_col
    
    nfl_team = NFL_clean(nfl_team)
    cities['NFL_teams_cleaned'] =  cities.apply(lambda x: NFL_clean(x['NFL_teams']), axis = 1)
    
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df.replace('[\*|\+]$', '', regex = True)
    nfl_df = nfl_df[['team', 'W', 'L']]
    numeric = nfl_df.apply(lambda x: x['W'].isnumeric(), axis = 1)
    nfl_df = nfl_df[numeric]
    nfl_df['team parsed'] = nfl_df.apply(lambda x: NFL_team_parse(x['team'], nfl_team), axis = 1)
    nfl_df = nfl_df.set_index('team parsed')
    
    # Get win-loss ratio
    cities['W_L ratio_NFL'] = cities.apply(lambda x: nfl_win_loss(x['NFL_teams_cleaned'], nfl_df), axis = 1)
    
    # Filter cities df
    cities = cities[cities['W_L ratio_NFL'] != False]

    return cities[['W_L ratio_NFL']]

In [65]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#mlb_df=pd.read_csv("assets/mlb.csv")
#nhl_df=pd.read_csv("assets/nhl.csv")
#nba_df=pd.read_csv("assets/nba.csv")
#nfl_df=pd.read_csv("assets/nfl.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    mlb_df=mlb_gen()
    nhl_df=nhl_gen()
    nba_df=nba_gen()
    nfl_df=nfl_gen()
    
    MLB_NHL = mlb.merge(nhl, how = 'inner', left_index = True, right_index = True)
    p_MLB_NHL = stats.ttest_rel(MLB_NHL['W_L ratio_MLB'], MLB_NHL['W_L ratio_NHL'])[1]
    
    MLB_NBA = mlb.merge(nba, how = 'inner', left_index = True, right_index = True)
    p_MLB_NBA = stats.ttest_rel(MLB_NBA['W_L ratio_MLB'], MLB_NBA['W_L ratio_NBA'])[1]
    
    MLB_NFL = mlb.merge(nfl, how = 'inner', left_index = True, right_index = True)
    p_MLB_NFL = stats.ttest_rel(MLB_NFL['W_L ratio_MLB'], MLB_NFL['W_L ratio_NFL'])[1]   
    
    NHL_NBA = nhl.merge(nba, how = 'inner', left_index = True, right_index = True)
    p_NHL_NBA = stats.ttest_rel(NHL_NBA['W_L ratio_NHL'], NHL_NBA['W_L ratio_NBA'])[1]
    
    NHL_NFL = nhl.merge(nfl, how = 'inner', left_index = True, right_index = True)
    p_NHL_NFL = stats.ttest_rel(NHL_NFL['W_L ratio_NHL'], NHL_NFL['W_L ratio_NFL'])[1] 

    NBA_NFL = nba.merge(nfl, how = 'inner', left_index = True, right_index = True)
    p_NBA_NFL = stats.ttest_rel(NBA_NFL['W_L ratio_NBA'], NBA_NFL['W_L ratio_NFL'])[1] 
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    p_values.loc["MLB", "NHL"] = p_MLB_NHL
    p_values.loc["NHL", "MLB"] = p_MLB_NHL
    
    p_values.loc["MLB", "NBA"] = p_MLB_NBA
    p_values.loc["NBA", "MLB"] = p_MLB_NBA
    
    p_values.loc["MLB", "NFL"] = p_MLB_NFL
    p_values.loc["NFL", "MLB"] = p_MLB_NFL  
    
    p_values.loc["NHL", "NBA"] = p_NHL_NBA
    p_values.loc["NBA", "NHL"] = p_NHL_NBA
    
    p_values.loc["NHL", "NFL"] = p_NHL_NFL
    p_values.loc["NFL", "NHL"] = p_NHL_NFL

    p_values.loc["NBA", "NFL"] = p_NBA_NFL
    p_values.loc["NFL", "NBA"] = p_NBA_NFL
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values